In [ ]:
%python
html = """<h1 style="color:Black;text-align:center;font-family:Ariel">Report on Dominant Airports</h1>"""
displayHTML(html)

In [ ]:
CREATE OR REPLACE TEMP VIEW v_dominant_airports
AS
SELECT origin_df.origin_airport AS airport,
        (origin_count+dest_count) AS count,
        RANK() OVER(ORDER BY (origin_count+dest_count) DESC) AS airport_rank
FROM ((SELECT origin_airport, COUNT(1) AS origin_count 
      FROM dev_air_travel_present.search_flight 
      GROUP BY origin_airport) AS origin_df
      INNER JOIN
      (SELECT dest_airport, COUNT(1) AS dest_count 
      FROM dev_air_travel_present.search_flight 
      GROUP BY dest_airport) AS dest_df
      ON origin_df.origin_airport = dest_df.dest_airport
     )
WHERE (origin_count+dest_count) >= 50
ORDER BY count DESC;

In [ ]:
SELECT origin_df.year,
        origin_df.origin_airport AS airport,
        (origin_count+dest_count) AS count
FROM ((SELECT year, origin_airport, COUNT(1) AS origin_count 
      FROM dev_air_travel_present.search_flight 
      GROUP BY year, origin_airport) AS origin_df
      INNER JOIN
      (SELECT year, dest_airport, COUNT(1) AS dest_count 
      FROM dev_air_travel_present.search_flight 
      GROUP BY year, dest_airport) AS dest_df
      ON origin_df.origin_airport = dest_df.dest_airport AND origin_df.year = dest_df.year
     )
WHERE origin_df.origin_airport IN (SELECT airport FROM v_dominant_airports WHERE airport_rank <= 5)
ORDER BY year, count DESC;

In [ ]:
%python
## Create Dashboard